In [12]:
#PostgreSQLで練習
import psycopg2 as psql

# PostgreSQLへ接続（接続情報をあなたの環境に合わせて変更）
    #因みに、データベース自体はターミナルで作った(gpt曰く、データベースは既存のものを利用するのが当たり前で、データベースを作れない人も多いためと出たため) 
conn = psql.connect(
    host="localhost",       # または IPアドレス(ここは自分のパソコンのpostgresqlに接続するときは変える必要がない)
    database="test_db",     # 作成したDB名
    user="koryosuzuki",        # ユーザー名
    password="56156956"  # PostgreSQLのパスワード
)
# このconn.cursorはカーソルを作るための関数そのものであり、conn.cursor()にすると、その関数を呼んで実際に実行するという意味になる。
    #つまり、(関数を実行した結果)カーソルを作りたいならば、()をつけなくてはいけない
cursor = conn.cursor()

# 既存テーブルを削除（あれば）
cursor.execute("DROP TABLE IF EXISTS employees;")
cursor.execute("DROP TABLE IF EXISTS departments;")

# 部署テーブル作成
cursor.execute("""
CREATE TABLE departments (
    dept_id INT PRIMARY KEY,
    dept_name VARCHAR(50) NOT NULL
);
""")

# 社員テーブル作成
cursor.execute("""
CREATE TABLE employees (
    emp_id INT PRIMARY KEY,
    emp_name VARCHAR(50) NOT NULL,
    age INT,
    salary INT,
    dept_id INT REFERENCES departments(dept_id),
    hire_date DATE
);
""")

# データ挿入（部署）
    #excutemanyは処理を何回も実行したいときに行う(insertで挿入できる行は一つだから、excutemanyを行わなくてはいけない)
        #SQLの本を使い、後でINSERT INTOを復習
cursor.executemany("""
INSERT INTO departments (dept_id, dept_name) VALUES (%s, %s);
""", [
    (1, "営業"),
    (2, "開発"),
    (3, "人事")
])

# データ挿入（社員）
    # まずは、200件のデータを作る
import random
# datetimeはモジュールでもありクラス名でもあるため、このような場合は、datetimeを呼び出しただけの場合、datetime.datetimeなどと書かなくてはいけない
    # 故にfromを使ってモジュールを定義し、datetimeとtimedeltaというクラス名を定義すると使いやすい。
from datetime import datetime, timedelta

first_names = ["田中", "佐藤", "鈴木", "高橋", "伊藤", "渡辺", "山本", "中村", "小林", "加藤"]
last_names = ["太郎", "花子", "一郎", "洋子", "誠", "真由美", "健太", "彩香", "翔太", "美咲"]

#def XXX:とした場合、XXXという関数を定義し、関数が呼ばれたときに:以下の処理をするということ
    # つまり、random_dateが呼ばれたら、(start + timedelta(days=random_days)).strftime("%Y-%m-%d")がリターンされるという処理が行われる
def random_date(start_year=2010, end_year=2023):
    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31)
    delta = end - start
    random_days = random.randint(0, delta.days)
    return (start + timedelta(days=random_days)).strftime("%Y-%m-%d")

#上記でdef random_dateと、関数が定義されているので、random_date()とすると、start_year=2010, end_year=2023のrandom_dateの計算が返される
    # rangeは(1,201)とすると、1~200までの数字を定義できる
data = []
for emp_id in range(1, 201):  # 200件分
    name = random.choice(first_names) + random.choice(last_names)
    age = random.randint(22, 60)
    salary = random.randint(3500000, 9000000)
    dept_id = random.randint(1, 3)  # 部署IDは1〜3に修正
    hire_date = random_date()
    data.append((emp_id, name, age, salary, dept_id, hire_date))
    # 次にこのデータを格納する
cursor.executemany("""
INSERT INTO employees (emp_id, emp_name, age, salary, dept_id, hire_date)
VALUES (%s, %s, %s, %s, %s, %s);
""", data)

# コミットして反映
conn.commit()
print("✅ データ作成完了")

# 接続終了
cursor.close()
conn.close()

✅ データ作成完了
